In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
from scipy.stats import skew
import statsmodels.api as sm

%matplotlib inline

In [2]:
ACCRE_df = pd.read_csv('data/fullsample.csv')

In [3]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [ ]:
ACCRE_df.info()
# Notice 4,365,379 entries

In [ ]:
ACCRE_df.tail()
# No footer to remove

In [4]:
ACCRE_df = ACCRE_df.drop_duplicates()

In [ ]:
ACCRE_df.shape

In [ ]:
# ACCRE_df_abridged = pd.read_csv('data/fullsample_dedup.csv')
# Reading in this csv is just to check if we eliminated the correct number of entries. 
# The ACCRE guy gave us this edited csv. 

In [5]:
ACCRE_df = ACCRE_df.loc[ACCRE_df['PARTITION'] == 'production']
ACCRE_df.head()
# We want to ignore the partition named "debug"

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [ ]:
ACCRE_df.shape

In [6]:
ACCRE_df.REQTIME = ACCRE_df.REQTIME.str.replace('-',':')
ACCRE_df.USEDTIME = ACCRE_df.USEDTIME.str.replace('-',':')
# REQTIME and USEDTIME are now manageable 

In [ ]:
ACCRE_df.head()

In [7]:
date_time=ACCRE_df.REQTIME

In [8]:
%%time
# REQTIME needs to be in total seconds.
to_seconds = []
for time in date_time:
    split=time.split(':')
    if len(split)==4:
        seconds=int(split[0])*86400+int(split[1])*3600+int(split[2])*60+int(split[3])
        to_seconds.append(seconds)
    else: 
        seconds=int(split[0])*3600+int(split[1])*60+int(split[2])
        to_seconds.append(seconds)

CPU times: user 5.94 s, sys: 103 ms, total: 6.04 s
Wall time: 6.27 s


In [9]:
ACCRE_df.REQTIME = to_seconds
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,1209600,11:01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,1188000,12:09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,1188000,13:08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,1188000,12:03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,1188000,11:08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [10]:
# used_time = ACCRE_df.USEDTIME

In [11]:
def convert_to_seconds(new):
    split_again=new.split(':')
    if len(split_again)==4: 
        sec_2=int(split_again[0])*86400+int(split_again[1])*3600+int(split_again[2])*60+int(split_again[3])
    
    else: 
        sec_2=int(split_again[0])*3600+int(split_again[1])*60+int(split_again[2])
               
    return sec_2

In [12]:
%%time 
ACCRE_df.USEDTIME = ACCRE_df.USEDTIME.apply(convert_to_seconds)

CPU times: user 5.77 s, sys: 224 ms, total: 5.99 s
Wall time: 5.9 s


In [ ]:
ACCRE_df.head()
# head=ACCRE_df.head()
# Just want to mess with a (very small) subset

In [13]:
ACCRE_df.REQMEM = ACCRE_df['REQMEM'].str.replace(r'(Mn|Mc)', r' \1')
ACCRE_df.USEDMEM = ACCRE_df['USEDMEM'].str.replace(r'(M)', r' \1')
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808 Mn,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288 Mn,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288 Mn,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288 Mn,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288 Mn,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [14]:
def convert_to_mc(x,y,z): 
    num_unit=x.split(' ')
    number=int(num_unit[0])
    unit=num_unit[1]
    if unit =='Mn': 
        mc_conversion=(number*y)/(z) 
        
    else: 
        mc_conversion=number 
    
    return mc_conversion

In [15]:
def m_to_mc(a,b,c): 
    if a =='0':
        mc=a 
    else: 
        split_it=a.split(' ')
        value=float(split_it[0])
        letter=split_it[1]
        mc=(value*b)/(c)
    
    
    return mc

In [16]:
%%time
ACCRE_df['REQMEM']=ACCRE_df.apply(lambda row: convert_to_mc(row['REQMEM'],row['NODES'],row['CPUS']),axis=1)

CPU times: user 1min 50s, sys: 1.92 s, total: 1min 52s
Wall time: 1min 54s


In [17]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,15488.0,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288.0,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288.0,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288.0,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288.0,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [18]:
# This is necessary now because we can't divide by zero later on 
ACCRE_df.loc[ACCRE_df['CPUS']==0]

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
1556155,9524253,kumquat,kate,16384.0,16383.34 M,86400,35114,24,0,production,0:0,2019-07-01T14:22:00


In [19]:
ACCRE_df.drop(ACCRE_df[ACCRE_df['CPUS']==0].index, inplace=True)

In [20]:
ACCRE_df.shape

(3319256, 12)

In [21]:
%%time
ACCRE_df['USEDMEM']=ACCRE_df.apply(lambda row: m_to_mc(row['USEDMEM'],row['NODES'],row['CPUS']),axis=1)

CPU times: user 2min 7s, sys: 2.86 s, total: 2min 10s
Wall time: 2min 17s


In [22]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,15488.0,7201.07,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288.0,6516.1,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288.0,0.09,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288.0,8333.82,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288.0,8304.03,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [23]:
ACCRE_df.to_csv('data/clean_ACCRE_data.csv', index=None)

# Began new notebook here 

In [ ]:
ACCRE_df.groupby('ACCOUNT').mean()

In [ ]:
ACCRE_df.groupby('ACCOUNT').describe()

In [ ]:
ACCRE_df.groupby('ACCOUNT').diff().describe()

In [ ]:
ACCRE_df.groupby('ACCOUNT').diff(axis=1)

In [ ]:
ACCRE_df['USEDMEM']=ACCRE_df['USEDMEM'].astype('float64')

In [ ]:
ACCRE_df['PCT_Actual'] = (ACCRE_df['USEDMEM'].div(ACCRE_df['REQMEM']).mul(100))

In [ ]:
ACCRE_df['REQMEM'] = ACCRE_df['REQMEM'].astype('int64')

In [ ]:
ACCRE_df['USEDMEM'] = ACCRE_df['USEDMEM'].astype('int64')

In [ ]:
ACCRE_df.loc(ACCRE_df['NODES']==1)
ACCRE_df.loc(ACCRE_df['EXITCODE']=='0:0')

In [ ]:
ACCRE_df.sort_values(by='PCT_Actual', ascending = False)
ACCRE_df.head()

In [ ]:
# Optimizing memory is more important for longer running jobs than shorter running
# jobs as the resources are tied up for longer. If jobs are weighted by runtime,
# what is the average percent of memory used of the requested memory for each group?
   # "Groups" are: 
    # short= <5min 
    # long= >=5 min 
# 5 min = 300 sec

In [ ]:
 ACCRE_df.tail()

## Let's just look at the first five rows 

In [ ]:
ACCRE_head = ACCRE_df.head()

In [ ]:
ACCRE_head['PCT_Actual'].mean()

In [ ]:
ACCRE_head['USEDTIME'].mean()

In [ ]:
ACCRE_head.groupby("JOBID")["PCT_Actual"].mean()

In [ ]:
#ACCRE_df.groupby('JOBID')('PCT_Actual').mean()

In [ ]:
# ACCRE_df['Weighted_Avg']= sum((ACCRE_df['PCT_Actual']).mul(ACCRE_df['USEDTIME']))/(sum(ACCRE_df['USEDTIME']))

In [ ]:
def wavg(group, avg_name, weight_name):

    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [ ]:
ACCRE_head.groupby("JOBID").apply(wavg, "PCT_Actual", "USEDTIME")

In [ ]:
ACCRE_tail = ACCRE_df.tail()

In [ ]:
ACCRE_tail.groupby("JOBID")["PCT_Actual"].mean()

In [ ]:
ACCRE_tail.groupby("JOBID").apply(wavg, "PCT_Actual", "USEDTIME")

In [ ]:
ACCRE_tail['Score']= ACCRE_tail.apply(wavg,"PCT_Actual", "USEDTIME")
# merge on dataframe? Use np.average instead! Don't need to add back to dataframe 

In [ ]:
ACCRE_tail